In [2]:
import numpy as np
from os.path import join as oj
import nibabel as nib
import matplotlib.pyplot as plt
#ctrlSubjBaseImage

In [ ]:
def saveMatrixToNifti( numpyArray, NIIout, addRandom=True, copyTo3Stacks = True):
    ### Current I assume this is a 2D array, if it is not I will exit.. later on we can try dealing with 3/4 d arrays
#    print numpyArray.shape
    
    if len(numpyArray.shape) != 2:
        print("ERROR-- only works for 2D array input currently")
        sys.exit()
    (height,width)  = numpyArray.shape
    ##    backgroundMode = ['zeroes','gaussian']
    # Create a placeholder array in the proper shape to hold the unsparsed data matrix
    #a = np.zeros( (1,   (width) * (height) ))
    niiMatrix = np.zeros((height,width,3))  ## make this into a 3-D Matrix

    if addRandom:
        ### This will add a random Gaussian matrix to each input array
        noiseData = np.random.normal(scale=100,size=(height,width))
        niiMatrix[:,:,0] = noiseData + numpyArray ## Copy the data from A into the nifti matrix
    else:
        niiMatrix[:,:,0] = numpyArray 
        
    if copyTo3Stacks:
        niiMatrix[:,:,1] =  niiMatrix[:,:,0]
        niiMatrix[:,:,2] =  niiMatrix[:,:,0]
    
    niiImg = nib.Nifti1Image(niiMatrix, np.eye(4))
    
    ### I need to reshape A as well
    niiImg.to_filename(NIIout)
    return True

In [ ]:
# myArray = np.ones((100,100)) * 100
### Hot Spot is currently a square, starting at 0,0 to 10,10
subjsPerGroup = 10
imageMatrixWidth, imageMatrixHeight = ( 100, 100)  ##Size of the entire image

## Size of the ROIs I spike into the image
roiWidth,roiHeight = (10,10)

ctrlRoiStarts = [0,40]  ## Assuming squares and starting at the same x,y coordinate to make it easier for now
testRoiStarts = [60,80]
outputDir = 'randomiseInputMatrices'

### Output Control Subjects
ctrlSubjBaseImage = np.ones((imageMatrixWidth,imageMatrixHeight))
testSubjBaseImage = np.ones((imageMatrixWidth,imageMatrixHeight))

for roiStart in ctrlRoiStarts:
    print(roiStart)
    ctrlSubjBaseImage[roiStart:(roiStart+roiWidth),roiStart:(roiStart+roiWidth)]  = 300

for i in range(10):
    ptID = "Ctrl%02d" % i
    saveMatrixToNifti(ctrlSubjBaseImage,oj(outputDir,"ctrl%02d.nii.gz" % i))

### Output Test Subjects
for roiStart in testRoiStarts:
    testSubjBaseImage[roiStart:(roiStart+roiWidth),roiStart:(roiStart+roiHeight)]  = 300


for i in range(10):
    ptID = "Test%02d" % i
    saveMatrixToNifti(testSubjBaseImage,oj(outputDir,"test%02d.nii.gz" % i))
    
imageMaskArray = np.ones((imageMatrixWidth,imageMatrixHeight))
saveMatrixToNifti(imageMaskArray,oj(outputDir,'cohortImageMask.nii.gz'),addRandom=False)

In [ ]:
from ipywidgets import IntSlider,interact

curImage = testSubjBaseImage


def addNoiseToImage(noiseScale,ptSlice):
    (height,width)  = curImage.shape
    noiseData = np.random.normal(scale=noiseScale,size=(height,width))

    randNoiseImg = curImage + noiseData
    plt.imshow(randNoiseImg)
    plt.show()

### Add another parameter where I can view the entire stack....    
    
    
interact(addNoiseToImage,noiseScale=IntSlider(min=0,value=50,max=200),
         ptSlice=IntSlider(min=0,max=20),
         continuous_update=False)

In [3]:
subjsPerGroup = 10
imageMatrixWidth, imageMatrixHeight = ( 25000, 25000)##Size of the entire image

## Size of the ROIs I spike into the image
roiWidth,roiHeight = (32,32)

ctrlRoiStarts = [0,40,440]  ## Assuming squares and starting at the same x,y coordinate to make it easier for now
testRoiStarts = [60,80,150]
outputDir = 'randomiseInputMatrices'

stackedOutputArray = np.array([])

### Output Control Subjects
ctrlSubjBaseImage = np.ones((imageMatrixWidth,imageMatrixHeight,1))
testSubjBaseImage = np.ones((imageMatrixWidth,imageMatrixHeight,1))

## Generate ROIs/Blobs for the control cohort
for roiStart in ctrlRoiStarts:
    ctrlSubjBaseImage[roiStart:(roiStart+roiWidth),roiStart:(roiStart+roiWidth),0]  = 300

### Output Test Subjects
for roiStart in testRoiStarts:
    testSubjBaseImage[roiStart:(roiStart+roiWidth),roiStart:(roiStart+roiHeight),0]  = 300

ptArrays = []
noiseScale = 50

for i in range(subjsPerGroup):
    noiseData = np.random.normal(scale=noiseScale,size=(imageMatrixWidth,imageMatrixHeight,1))
    ptArrays.append( ctrlSubjBaseImage+noiseData )

for i in range(subjsPerGroup):
    noiseData = np.random.normal(scale=noiseScale,size=(imageMatrixWidth,imageMatrixHeight,1))
    ptArrays.append( testSubjBaseImage+noiseData )
    
stackedOutputArray =  np.stack(ptArrays,axis=-1)

In [ ]:
from ipywidgets import IntSlider,interact
curImage = stackedOutputArray

def showPtSlice(ptSlice):
    plt.imshow(stackedOutputArray[:,:,0,ptSlice])
    plt.show()

### Add another parameter where I can view the entire stack....    
interact(showPtSlice,ptSlice=IntSlider(min=0,max=19,continuous_update=False))

In [ ]:
niiImg = nib.Nifti1Image(stackedOutputArray, np.eye(4))
niiImg.to_filename( oj(outputDir,'sampleNii20PtImage.nii.gz'))


imageMaskArray = np.ones((imageMatrixWidth,imageMatrixHeight,1))
niiMaskImg = nib.Nifti1Image(imageMaskArray,np.eye(4))
niiMaskImg.to_filename(oj(outputDir,'cohortImageMask.nii.gz'))
